In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,72.16,78,15,3.76,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,72.55,98,24,9.19,few clouds
2,2,Yellowknife,CA,62.4560,-114.3525,46.92,71,75,1.99,broken clouds
3,3,Kapaa,US,22.0752,-159.3190,84.18,71,40,4.00,scattered clouds
4,4,Busselton,AU,-33.6500,115.3333,61.05,59,51,9.46,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature that you would accept for your trip?'))
max_temp = float(input('What is the maximum temperature that you would accept for your trip?'))

What is the minimum temperature that you would accept for your trip?60
What is the maximum temperature that you would accept for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,72.16,78,15,3.76,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,72.55,98,24,9.19,few clouds
3,3,Kapaa,US,22.0752,-159.3190,84.18,71,40,4.00,scattered clouds
4,4,Busselton,AU,-33.6500,115.3333,61.05,59,51,9.46,broken clouds
9,9,Bengkulu,ID,-3.8004,102.2655,83.35,67,54,3.69,broken clouds
10,10,Port Alfred,ZA,-33.5906,26.8910,62.11,36,0,8.59,clear sky
11,11,Malkara,TR,40.8900,26.9011,61.83,61,98,1.95,overcast clouds
12,12,Tigzirt,DZ,36.8886,4.1198,64.54,89,0,2.30,clear sky
13,13,Solton,RU,52.8429,86.4787,66.83,61,25,2.33,scattered clouds
15,15,Bambous Virieux,MU,-20.3428,57.7575,76.21,78,75,18.37,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                465
City                   465
Country                463
Lat                    465
Lng                    465
Max Temp               465
Humidity               465
Cloudiness             465
Wind Speed             465
Current Description    465
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                463
City                   463
Country                463
Lat                    463
Lng                    463
Max Temp               463
Humidity               463
Cloudiness             463
Wind Speed             463
Current Description    463
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,72.16,few clouds,42.6001,-73.9662,
1,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,
3,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
4,Busselton,AU,61.05,broken clouds,-33.6500,115.3333,
9,Bengkulu,ID,83.35,broken clouds,-3.8004,102.2655,
10,Port Alfred,ZA,62.11,clear sky,-33.5906,26.8910,
11,Malkara,TR,61.83,overcast clouds,40.8900,26.9011,
12,Tigzirt,DZ,64.54,clear sky,36.8886,4.1198,
13,Solton,RU,66.83,scattered clouds,52.8429,86.4787,
15,Bambous Virieux,MU,76.21,light rain,-20.3428,57.7575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')

hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,72.16,few clouds,42.6001,-73.9662,
1,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Busselton,AU,61.05,broken clouds,-33.6500,115.3333,Observatory Guest House
9,Bengkulu,ID,83.35,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,72.16,few clouds,42.6001,-73.9662,
1,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Busselton,AU,61.05,broken clouds,-33.6500,115.3333,Observatory Guest House
9,Bengkulu,ID,83.35,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
10,Port Alfred,ZA,62.11,clear sky,-33.5906,26.8910,The Halyards Hotel
11,Malkara,TR,61.83,overcast clouds,40.8900,26.9011,Ütğm Korel Lojmanı
12,Tigzirt,DZ,64.54,clear sky,36.8886,4.1198,Bloc D tazoumbirt (amar)
13,Solton,RU,66.83,scattered clouds,52.8429,86.4787,
15,Bambous Virieux,MU,76.21,light rain,-20.3428,57.7575,Casa Tia Villa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Call the figure to plot the data.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
fig

Figure(layout=FigureLayout(height='420px'))